# Notebook to delete an AWS Redshift cluster

In [1]:
import pandas as pd
import boto3
import json

#### Set verbosity to 0|1|2 (default=0)

In [2]:
VERBOSE = 1

## STEP-1:  Load cluster parameters from `myDWH.cfg`

In [3]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('myDWH.cfg'))

# Retrieve the USER-related  parameters
USR_KEY                = config.get('USR','USR_KEY')
USR_SECRET             = config.get('USR','USR_SECRET')

# Retrieve the AWS-related  parameters
AWS_REGION             = config.get('AWS','AWS_REGION')

# Retrieve the CLUSTER-related  parameters
CLUSTER_NAME           = config.get("CLUSTER","CLUSTER_NAME")
CLUSTER_TYPE           = config.get("CLUSTER","CLUSTER_TYPE")
CLUSTER_NODE_TYPE      = config.get("CLUSTER","CLUSTER_NODE_TYPE")
CLUSTER_NODE_COUNT     = config.get("CLUSTER","CLUSTER_NODE_COUNT")

CLUSTER_DB_NAME        = config.get("CLUSTER","CLUSTER_DB_NAME")
CLUSTER_DB_USER        = config.get("CLUSTER","CLUSTER_DB_USER")
CLUSTER_DB_PASSWORD    = config.get("CLUSTER","CLUSTER_DB_PASSWORD")
CLUSTER_DB_PORT        = config.get("CLUSTER","CLUSTER_DB_PORT")

# Retrieve the IAM-related parameters
AWS_RESOURCE_NAME      = config.get("IAM_ROLE", "AWS_RESOURCE_NAME")

(CLUSTER_DB_USER, CLUSTER_DB_PASSWORD, CLUSTER_DB_NAME)

if VERBOSE > 0:
    df = pd.DataFrame({"Param":
                       ["CLUSTER_TYPE", "CLUSTER_NODE_COUNT", "CLUSTER_NODE_TYPE", "CLUSTER_NAME", "CLUSTER_DB_NAME", "CLUSTER_DB_USER", "CLUSTER_DB_PASSWORD", "CLUSTER_DB_PORT", "AWS_RESOURCE_NAME", "AWS_REGION"],
                       "Value":
                       [ CLUSTER_TYPE ,  CLUSTER_NODE_COUNT ,  CLUSTER_NODE_TYPE ,  CLUSTER_NAME ,  CLUSTER_DB_NAME ,  CLUSTER_DB_USER ,  CLUSTER_DB_PASSWORD ,  CLUSTER_DB_PORT ,  AWS_RESOURCE_NAME ,  AWS_REGION ]
                      })
    print(df)

                 Param       Value
0         CLUSTER_TYPE  multi-node
1   CLUSTER_NODE_COUNT           2
2    CLUSTER_NODE_TYPE   dc2.large
3         CLUSTER_NAME  dwhCluster
4      CLUSTER_DB_NAME         dwh
5      CLUSTER_DB_USER     dwhuser
6  CLUSTER_DB_PASSWORD    Passw0rd
7      CLUSTER_DB_PORT        5439
8    AWS_RESOURCE_NAME     dwhRole
9           AWS_REGION   us-west-2


## STEP-2: Create a client for Redshift and IAM

In [4]:
if VERBOSE > 0:
    print("Create clients for Redshift and IAM")

redshift = boto3.client('redshift',
                       region_name=AWS_REGION,
                       aws_access_key_id=USR_KEY,
                       aws_secret_access_key=USR_SECRET
                       )

iam = boto3.client('iam',aws_access_key_id=USR_KEY,
                     aws_secret_access_key=USR_SECRET,
                     region_name=AWS_REGION
                  )

Create clients for Redshift and IAM


## STEP-3: Clean up the running cluster

<b><font color='red'>DO NOT RUN THIS UNLESS YOU ARE SURE <br/>

In [5]:
#### CAREFUL - This delete the created resources
redshift.delete_cluster( ClusterIdentifier=CLUSTER_NAME,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

{'Cluster': {'ClusterIdentifier': 'dwhcluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'MasterUsername': 'dwhuser',
  'DBName': 'dwh',
  'Endpoint': {'Address': 'dwhcluster.c4p6b3uqdbp8.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2023, 6, 8, 16, 44, 33, 239000, tzinfo=tzlocal()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-001ef50f88654cbd0',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-040a5b8903624d5ca',
  'AvailabilityZone': 'us-west-2d',
  'PreferredMaintenanceWindow': 'mon:08:00-mon:08:30',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 2,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [],
  'EnhancedVpcRouti

## STEP-4: Wait for the cluster to be deleted

- Run this block several times until the cluster really deleted

In [6]:
import time

try:
    while redshift.describe_clusters(ClusterIdentifier=CLUSTER_NAME)['Clusters'][0]['ClusterStatus'] != 'available':
        if VERBOSE > 0:
            print("Cluster is being deleted...")
        time.sleep(10)
except Exception as e:
    if VERBOSE > 0:
        print(e)

print('Cluster is deleted')


Cluster is being deleted...
Cluster is being deleted...
Cluster is being deleted...
Cluster is being deleted...
Cluster is being deleted...
Cluster is being deleted...
Cluster is being deleted...
An error occurred (ClusterNotFound) when calling the DescribeClusters operation: Cluster dwhcluster not found.
Cluster is deleted


## STEP-5: Detach the IAM Role and Policy

In [7]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=AWS_RESOURCE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=AWS_RESOURCE_NAME)
#### CAREFUL!!

{'ResponseMetadata': {'RequestId': 'c0562625-92eb-4b31-8870-e055b6c1a5c2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c0562625-92eb-4b31-8870-e055b6c1a5c2',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Thu, 08 Jun 2023 22:15:56 GMT'},
  'RetryAttempts': 0}}